In [76]:
#read triples from training set
training_f = 'kbc/src_data/FB237/original/train'
train_triples = []
with open(training_f, 'r') as f:
    while True:
        line = f.readline()
        if line:
            # two relations split by ',', confidence split by tab
            line.replace('\n', '')
            lhs = (line.split('\t')[0])
            rel = (line.split('\t')[1])
            rhs = (line.split('\t')[2])
            
#             if rel in rel_head_idx:
            train_triples.append((lhs, rel, rhs))
        else:
            break
print(len(train_triples))

valid_f = 'kbc/src_data/FB237/original/valid'
valid_triples = []
with open(valid_f, 'r') as f:
    while True:
        line = f.readline()
        if line:
            # two relations split by ',', confidence split by tab
            line.replace('\n', '')
            lhs = (line.split('\t')[0])
            rel = (line.split('\t')[1])
            rhs = (line.split('\t')[2])
            
#             if rel in rel_head_idx:
            valid_triples.append((lhs, rel, rhs))
        else:
            break
print(len(valid_triples))

test_f = 'kbc/src_data/FB237/original/test'
test_triples = []
with open(test_f, 'r') as f:
    while True:
        line = f.readline()
        if line:
            # two relations split by ',', confidence split by tab
            line.replace('\n', '')
            lhs = (line.split('\t')[0])
            rel = (line.split('\t')[1])
            rhs = (line.split('\t')[2])
            
#             if rel in rel_head_idx:
            test_triples.append((lhs, rel, rhs))
        else:
            break
print(len(test_triples))

print(len(set(train_triples + valid_triples)))

272115
17535
20466
289650


In [77]:
from collections import defaultdict
# read in all rules (using non-circular rules for now)
rule_path = 'kbc/src_data/FB237/original/_entailment_cons.txt'
#### read in & convert rule ids and confidence
kbc_id_conf_f = rule_path
r_p_list = []
r_q_list = []
conf_list = []
neg_list = []
r_q_dict = defaultdict(list)
with open(kbc_id_conf_f, 'r') as f:
    while True:
        line = f.readline()
        if line:
            # two relations split by ',', confidence split by tab
            line.replace('\n', '')
            if '-' in line:
                neg_list.append(-1)
                line = line[1:]
            else:
                neg_list.append(1)
            r_p = line.split(',')[0]
            r_q = line.split(',')[1].split('\t')[0]
            conf = line.split('\t')[1]
            # print(rel0, rel1, conf)
            r_p_list.append(r_p)
            r_q_list.append(r_q)
            conf_list.append(conf)
            r_q_dict[r_q].append(r_p)
        else:
            break
rule_list = [r_p_list, r_q_list, conf_list, neg_list]
r_set = set(r_p_list + r_q_list)
print(len(rule_list[0]))
print(len(r_set))
print(len(set(r_p_list) - set(r_q_list)))

76
55
5


# Transductive experiments:
## Keep only some triplets of all head relations:
1. For all rp, in training set, keep only one (e1, rp, e2) for all (e1, rp, _)
2. For rp → rq, if (e1, rp, e2), Remove: (e1, rq, _) and (_, rq, e2), (e2, rq, e1)

In [34]:
tmp_triples = train_triples.copy()
print(len(train_triples))
print(len(tmp_triples))
seen_dict = {}
cnt = 0
for e1, r, e2 in train_triples:
    for (r_p, r_q) in zip(r_p_list, r_q_list):
        if r == r_p and r_p != r_q:
            if (e1, r) in seen_dict:
                try:
                    tmp_triples.remove((e1, r, e2))
                    break
                except: 
                    pass
            else:
                seen_dict[(e1, r)] = (e1, r, e2)
                break
print(len(tmp_triples))
train_triples = tmp_triples

272115
272115
264964


In [ ]:
len(train_triples)
new_train_file = open('kbc/src_data/FB237/train_1004', "w")
for e1, r, e2 in train_triples:
    new_train_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_train_file.close()

In [ ]:
# filter out some triples with head relations
entailment_triples = set()
remove_triples = []
test_triples = set()
head_set = set()
tail_set = set()
cnt = 0
# for e1, r, e2 in train_triples:
#     if cnt % 5000 == 0:
#         print(cnt)
#     cnt += 1
#     for (r_p, r_q) in zip(r_p_list, r_q_list):
#         if r == r_p and r_p != r_q:
#             test_triples.append((e1, r_q, e2))
# #             print(e1, r, e2, r_p, r_q)
#             tmp = [i for i in train_triples if (i[0] == e1 and i[1] == r_q and i[2] != e2) or (i[0] != e1 and i[1] == r_q and i[2] == e2) or (i[0] == e2 and i[1] == r_q and i[2] != e1) or (i[0] != e2 and i[1] == r_q and i[2] == e1)]
#             if len(tmp): 
# #                 print((tmp[0]))
#                 remove_triples += tmp
    
# remove_triples = list(set(remove_triples))
# print("extraction done")
# entailment_triples = [i for i in train_triples if i not in remove_triples]
# print(len(entailment_triples))
# print(len(test_triples))

for e1, r, e2 in train_triples:
    dup = 0
    for (r_p, r_q) in zip(r_p_list, r_q_list):
        if r == r_p and r_p != r_q:
            test_triples.add((e1, r_q, e2))
            if ((e1, r) in head_set) or ((r, e2) in tail_set):
                # duplicate, skip
                dup = 1
                break
            else:
                entailment_triples.add((e1, r, e2))
                head_set.add((e1, r))
                tail_set.add((r, e2))
    if not dup:
        entailment_triples.add((e1, r, e2))
    
# remove_triples = list(set(remove_triples))
print("extraction done")
# entailment_triples = [i for i in train_triples if i not in remove_triples]
entailment_triples = list(entailment_triples)
print(len(entailment_triples))
print(len(test_triples))

# filter out some triples with head relations from valid set
entailment_valid_triples = set()
remove_valid_triples = []
head_set = set()
tail_set = set()
cnt = 0
for e1, r, e2 in valid_triples:
    dup = 0
    for (r_p, r_q) in zip(r_p_list, r_q_list):
        if r == r_p and r_p != r_q:
            if ((e1, r) in head_set) or ((r, e2) in tail_set):
                # duplicate, skip
                dup = 1
                break
            else:
                head_set.add((e1, r))
                tail_set.add((r, e2))
                entailment_valid_triples.add((e1, r, e2))
    if not dup:
        entailment_valid_triples.add((e1, r, e2))

entailment_valid_triples = list(entailment_valid_triples)
print(len(entailment_valid_triples))

extraction done
261149
4547
17500


In [36]:
# len(remove_triples)
test_triples = list(set(test_triples))
len(test_triples)

4547

In [37]:
# save triples to file
new_train_file = open('kbc/src_data/FB237/train_1004', "w")
for e1, r, e2 in entailment_triples:
    new_train_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_train_file.close()

new_valid_file = open('kbc/src_data/FB237/valid_1004', "w")
for e1, r, e2 in tmp_valid_triples:
    new_valid_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_valid_file.close()

new_test_file = open('kbc/src_data/FB237/test_1004', "w")
for e1, r, e2 in test_triples:
    new_test_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_test_file.close()

# Inductive experiments:
Randomly select from test set, omit any tuples from the training file with the entity in the new test file

In [20]:
#read triples from test set
test_f = 'kbc/src_data/FB237/original/test'
test_triples = []
with open(test_f, 'r') as f:
    while True:
        line = f.readline()
        if line:
            # two relations split by ',', confidence split by tab
            line.replace('\n', '')
            lhs = (line.split('\t')[0])
            rel = (line.split('\t')[1])
            rhs = (line.split('\t')[2])
            
            test_triples.append((lhs, rel, rhs))
        else:
            break
print(len(test_triples))

20466


In [21]:
# randomly select 30% of triples, extract entities
import random
sample_num = len(test_triples) * 20 // 100
sample_test_triples = random.sample(test_triples, sample_num)
print(len(sample_test_triples))
ent_set = set()
for triple in sample_test_triples:
    ent_set.add(triple[0])
    ent_set.add(triple[2])
print(len(ent_set))

4093
5256


In [23]:
# omit entities from training and valid set
tmp_triples = train_triples.copy()
for triple in train_triples:
    if triple[0] in ent_set or triple[2] in ent_set:
        tmp_triples.remove(triple)
print(len(tmp_triples))
# train_triples = tmp_triples

tmp_valid_triples = valid_triples.copy()
for triple in valid_triples:
    if triple[0] in ent_set or triple[2] in ent_set:
        tmp_valid_triples.remove(triple)
print(len(tmp_valid_triples))

79778
3549


In [24]:
# save triples to file
new_train_file = open('kbc/src_data/FB237/train_1007', "w")
for e1, r, e2 in tmp_triples:
    new_train_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_train_file.close()

new_valid_file = open('kbc/src_data/FB237/valid_1007', "w")
for e1, r, e2 in tmp_valid_triples:
    new_valid_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_valid_file.close()

new_test_file = open('kbc/src_data/FB237/test_1007', "w")
for e1, r, e2 in sample_test_triples:
    new_test_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_test_file.close()

# Synthesized experiments(10.14):
- For (e1, r, e2) in test and r in r_q set, we don't want (e1, r, _), (_, r, e2) to be seen in training and valid set, but we still want e1, r, e2, (e1, r_p, e2) to be seen

In [7]:
# remove unrelevant test triples
new_test_triples = []
r_q_set = set(r_q_list)
print(len(r_q_set))
for e1, r, e2 in test_triples:
    if r in r_q_set:
        new_test_triples.append((e1, r, e2))
print(len(new_test_triples))

50
719


In [10]:
head_set = set()
tail_set = set()
extract_train_triples = set()
extract_valid_triples = set()
for e1, r, e2 in test_triples:
    for (r_p, r_q) in zip(r_p_list, r_q_list):
        if r == r_q:
            head_set.add((e1, r))
#             head_set.add((e1, r_p))
            tail_set.add((r, e2))
#             tail_set.add((r_p, e2))
print(len(head_set))
print(len(tail_set))
for e1, r, e2 in train_triples:
    if (e1, r) not in head_set and (r, e2) not in tail_set:
        extract_train_triples.add((e1, r, e2))
print(len(extract_train_triples))

for e1, r, e2 in valid_triples:
    if (e1, r) not in head_set and (r, e2) not in tail_set:
        extract_valid_triples.add((e1, r, e2))
print(len(extract_valid_triples))

# add tail triples to training and valid sets
for e1, r, e2 in new_test_triples:
    for r_p in r_q_dict[r]:
        extract_valid_triples.add((e1, r_p, e2))
        extract_triples.add((e1, r_p, e2))
print(len(extract_triples))
print(len(extract_valid_triples))

650
635
252430
17336
253220
18103


In [11]:
# save triples to file
new_train_file = open('kbc/src_data/FB237/train_1014', "w")
for e1, r, e2 in extract_train_triples:
    new_train_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_train_file.close()

new_valid_file = open('kbc/src_data/FB237/valid_1014', "w")
for e1, r, e2 in extract_valid_triples:
    new_valid_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_valid_file.close()

new_test_file = open('kbc/src_data/FB237/test_1014', "w")
for e1, r, e2 in new_test_triples:
    new_test_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_test_file.close()

In [49]:
print(len(test_triples))
print(len(r_set))

20466
55


In [53]:
new_test_file = open('kbc/src_data/FB237/test_1014', "w")
for e1, r, e2 in new_test_triples:
    new_test_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_test_file.close()

# Synthesized experiments(11.14):
- Create "low_freq relation/entity" test set
    - for all r_q, extract all triples from training & test set
    - add all such triples to test set (remove redundent), keep only 1 (is this enough?) r_q triple in training set (let the model learn r_q, low_freq, solve the lack of test triple problem)
    - only have **8 rules** (r_q not in r_p set)
- Validate effect of r_q freq in training set (12.7)
    - add all such triples to test set (remove redundent), sample different numbers (1, 2, 4, 8) of r_q triples into training set (let the model learn r_q, low_freq, solve the lack of test triple problem)

In [78]:
from collections import defaultdict
# read in all rules (using rules that r_q are not as r_p as well)
rule_path = 'kbc/src_data/FB237/original/_entailment_cons-rq.txt'
#### read in & convert rule ids and confidence
kbc_id_conf_f = rule_path
r_p_list = []
r_q_list = []
conf_list = []
neg_list = []
r_q_dict = defaultdict(list)
with open(kbc_id_conf_f, 'r') as f:
    while True:
        line = f.readline()
        if line:
            # two relations split by ',', confidence split by tab
            line.replace('\n', '')
            if '-' in line:
                neg_list.append(-1)
                line = line[1:]
            else:
                neg_list.append(1)
            r_p = line.split(',')[0]
            r_q = line.split(',')[1].split('\t')[0]
            conf = line.split('\t')[1]
            # print(rel0, rel1, conf)
            r_p_list.append(r_p)
            r_q_list.append(r_q)
            conf_list.append(conf)
            r_q_dict[r_q].append(r_p)
        else:
            break
rule_list = [r_p_list, r_q_list, conf_list, neg_list]
r_set = set(r_p_list + r_q_list)
print(len(rule_list[0]))
print(len(r_set))
print(len(set(r_p_list) - set(r_q_list)))

10
16
8


In [112]:
# extract all relevant triples in original train, valid and test data
# keep sample_num for each relation at training and valid
r_q_set = set(r_q_list) - set(r_p_list)
# print(r_q_set)
r_dict = defaultdict(list)
for e1, r, e2 in train_triples + test_triples + valid_triples:
    if r in r_q_set:
        r_dict[r].append((e1, r, e2))
# sample for each relation r_q
from random import sample
train_num = 1
test_num = 80
extracted_test_triples = []
cand_triples = []
all_r_q_triples = set()
for r in r_dict:
    all_r_q_triples |= set(r_dict[r])
    train_r = sample(r_dict[r], train_num)
    cand_triples += train_r
    test_r = list(set(r_dict[r]) - set(train_r))
    extracted_test_triples += test_r
print(len(extracted_test_triples))
print(len(cand_triples))
print(len(all_r_q_triples))

2245
8
2253


In [113]:
# all r_q relevant triples should be removed from original training and testing
extract_train_triples = set()
extract_valid_triples = set()
for t in train_triples:
    if t not in all_r_q_triples:
        extract_train_triples.add(t)
extract_train_triples = extract_train_triples | set(cand_triples)
print(len(extract_train_triples))

for t in valid_triples:
    if t not in all_r_q_triples:
        extract_valid_triples.add(t)
extract_valid_triples |= set(cand_triples)
print(len(extract_valid_triples))

269915
17521


In [95]:
# use for all sampling tests
static_extracted_test_triples = list(extracted_test_triples)
print(len(static_extracted_test_triples))

2189


In [114]:
# save triples to file
new_train_file = open('kbc/src_data/FB237/train_1207_1', "w")
for e1, r, e2 in extract_train_triples:
    new_train_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_train_file.close()

new_valid_file = open('kbc/src_data/FB237/valid_1207_1', "w")
for e1, r, e2 in extract_valid_triples:
    new_valid_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_valid_file.close()

new_test_file = open('kbc/src_data/FB237/test_1207', "w")
for e1, r, e2 in static_extracted_test_triples:
    new_test_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_test_file.close()

# Validation experiment(12.7):
- Create "validation" test set to check the correctness of model on low freq training setting (synthesized setting 11.7)
    - remove all r_q relevant triples from training & valid set
    - for all <e1, rp, e2> in training set, add <e1, rq, e2> into test set

In [74]:
import collections
valid_test_triples = set()
r_p_set = set(r_p_list)
r_p_dict = collections.defaultdict(list)
for r_p, r_q in zip(r_p_list, r_q_list):
    r_p_dict[r_p].append(r_q)
# print(r_p_dict)
for e1, r, e2 in (extract_train_triples | extract_valid_triples):
    if r in r_p_dict:
        for r_q in r_p_dict[r]:
            valid_test_triples.add((e1, r_q, e2))
print(len(valid_test_triples))

10224


In [75]:
# save triples to file
new_train_file = open('kbc/src_data/FB237/train_eval_1207', "w")
for e1, r, e2 in extract_train_triples:
    new_train_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_train_file.close()

new_valid_file = open('kbc/src_data/FB237/valid_eval_1207', "w")
for e1, r, e2 in extract_valid_triples:
    new_valid_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_valid_file.close()

new_test_file = open('kbc/src_data/FB237/test_eval_1207', "w")
for e1, r, e2 in valid_test_triples:
    new_test_file.write(str(e1) + '\t' + str(r) + '\t' + str(e2))
new_test_file.close()